In [ ]:
model = DetrForObjectDetection.from_pretrained(MODEL_PATH)

# Get Image
image_ids = test_set.coco.getImgIds()
image_id = random.choice(image_ids)
image = test_set.coco.loadImgs(image_id)[0]

# Load Image
img_path = os.path.join(imgs_path, image["file_name"])
image = cv2.imread(img_path)

# Plor Ground Truth
#plot_coco_image(coco_dataset=test_set, imgs_path = "../ExDark_All/Images", image_id=image_id)
    

In [ ]:
# Predict
with torch.no_grad():

    # load image and predict
    inputs = image_processor(images=image, return_tensors='pt')
    outputs = model(**inputs)

    # post-process
    target_sizes = torch.tensor([image.shape[:2]])
    results = image_processor.post_process_object_detection(
        outputs=outputs, 
        threshold=0.001, 
        target_sizes=target_sizes
    )[0]

    # Keep best predictions
    max_id = np.argmax(results["scores"])
    label = results['labels'][max_id]
    print(results)
    box = np.array(results["boxes"][max_id].unsqueeze(0))[0]
    bbox = [int(b) for b in box]
    image = Image.fromarray(image)
    draw = ImageDraw.Draw(image)
    draw.rectangle([bbox[0], bbox[1], bbox[0]+bbox[2], bbox[1]+bbox[3]], width=3)
    plt.imshow(image)
    plt.show()